In [1]:
import requests
import io

from datetime import date, timedelta

url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-15,France,108561.0,0.0,16643.0,0.0,0.0,0.0
0,2020-04-15,China,83515.0,0.0,3345.0,0.0,0.0,0.0
0,2020-04-15,Italy,165318.0,0.0,21614.0,0.0,0.0,0.0
0,2020-04-15,Spain,174669.0,0.0,18471.0,0.0,0.0,0.0
0,2020-04-15,United States,629948.0,0.0,28329.0,0.0,0.0,0.0
0,2020-04-15,World,2011184.0,0.0,131848.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")

In [6]:
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-16,France,108561.0,106206.0,16643.0,17167.0,2355.0,-524.0
0,2020-04-16,China,83515.0,83402.0,3345.0,3346.0,113.0,-1.0
0,2020-04-16,Italy,165318.0,165155.0,21614.0,21647.0,163.0,-33.0
0,2020-04-16,Spain,174669.0,177633.0,18471.0,18579.0,-2964.0,-108.0
0,2020-04-16,United States,629948.0,639664.0,28329.0,30985.0,-9716.0,-2656.0
0,2020-04-16,World,2011184.0,2029930.0,131848.0,136320.0,-18746.0,-4472.0


In [7]:
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")